# Apple Playstore reviews for MariBank

## Installing Packages

In [6]:
%%capture
! pip install app_store_scraper
! pip install emoji
! pip install langdetect
import pandas as pd
import numpy as np
import json
import emoji 
import re
from langdetect import detect

## Getting Data 

In [9]:
#link: https://apps.apple.com/sg/app/maribank/id1658919834

from app_store_scraper import AppStore
maribank = AppStore(country='sg', app_name='maribank', app_id = '1658919834')

# 22 reviews in total
maribank.review(how_many = 22) 

2024-03-27 10:33:51,826 [INFO] Base - Initialised: AppStore('sg', 'maribank', 1658919834)
2024-03-27 10:33:51,826 [INFO] Base - Ready to fetch reviews from: https://apps.apple.com/sg/app/maribank/id1658919834
2024-03-27 10:33:51,926 [INFO] Base - [id:1658919834] Fetched 21 reviews (21 fetched in total)


### Preprocessing Data 

In [16]:
data = maribank.reviews
maribank_apple_df = pd.DataFrame.from_records(data)
# Adding labels
maribank_apple_df['Bank'] = 'Maribank'
# Selecting Columns
maribank_apple_df = maribank_apple_df.reindex(columns=['userName', 'review', 'rating', 'date', 'Bank'])
# Reorder
maribank_apple_df = maribank_apple_df.reindex(columns=['userName', 'review', 'rating', 'date', 'Bank'])
# Rename
#maribank_apple_df.columns = ['UserName', 'Review', 'Score', 'Date', 'Bank']

maribank_apple_df

,userName,review,rating,date,Bank
0,felthebest,I liked the idea where its a online bank witho...,5,2024-01-06 16:07:27,Maribank
1,gniynew,The onboarding process was really quick and st...,5,2023-03-08 11:39:10,Maribank
2,boonnie38,I have to admit that the process to register i...,5,2024-01-04 06:21:44,Maribank
3,Elijahdptty,Finally a mobile banking app that doesn’t lag ...,5,2023-04-05 16:08:04,Maribank
4,ttyto12,"Overall good and smooth experience, enjoy the ...",5,2023-10-06 04:29:57,Maribank
5,ngtk,I am so glad you don’t have to resort to fake ...,5,2024-01-18 10:43:33,Maribank
6,LittleTroll,Easy to use and navigate. Everything is laid o...,5,2023-07-27 13:33:17,Maribank
7,Unlimeg,It would be good if there is a prompt to allow...,1,2023-12-09 09:29:10,Maribank
8,97.z,Instant approval and so hassle free to earn de...,5,2023-03-09 15:30:23,Maribank
9,DDYY1990,"Simple and good interest rate, no string attac...",5,2023-03-17 07:33:14,Maribank


In [18]:
# Convert emoji to words
def convert_emojis_to_words(text):
    converted_text = emoji.demojize(text)
    return converted_text

maribank_apple_df['review'] = maribank_apple_df['review'].apply(convert_emojis_to_words)
maribank_apple_df

,userName,review,rating,date,Bank
0,felthebest,I liked the idea where its a online bank witho...,5,2024-01-06 16:07:27,Maribank
1,gniynew,The onboarding process was really quick and st...,5,2023-03-08 11:39:10,Maribank
2,boonnie38,I have to admit that the process to register i...,5,2024-01-04 06:21:44,Maribank
3,Elijahdptty,Finally a mobile banking app that doesn’t lag ...,5,2023-04-05 16:08:04,Maribank
4,ttyto12,"Overall good and smooth experience, enjoy the ...",5,2023-10-06 04:29:57,Maribank
5,ngtk,I am so glad you don’t have to resort to fake ...,5,2024-01-18 10:43:33,Maribank
6,LittleTroll,Easy to use and navigate. Everything is laid o...,5,2023-07-27 13:33:17,Maribank
7,Unlimeg,It would be good if there is a prompt to allow...,1,2023-12-09 09:29:10,Maribank
8,97.z,Instant approval and so hassle free to earn de...,5,2023-03-09 15:30:23,Maribank
9,DDYY1990,"Simple and good interest rate, no string attac...",5,2023-03-17 07:33:14,Maribank


In [21]:
# Shows which reviews are not in english
maribank_apple_df['review'].apply(detect).eq('en')
# Checking the review is not in english

0      True
1      True
2      True
3      True
4      True
5      True
6      True
7      True
8      True
9      True
10     True
11     True
12     True
13     True
14     True
15     True
16     True
17     True
18     True
19     True
20    False
Name: review, dtype: bool

In [22]:
# converting it to csv
maribank_apple_df.to_csv('maribank_apple_df.csv')
